In [ ]:
using DataFrames, CSV, Plots
amzn = CSV.read(".../AMZN.csv", DataFrame)
p1 = plot(amzn."Date", amzn."Adj Close", title="Original Time Series
Data", legend=:topright)
using Statistics
function min_max_scaling(column)
    min_val = minimum(column)
    max_val = maximum(column)
    return (column .- min_val) ./ (max_val .- min_val)
end
amzn_norm = copy(amzn)
for col in names(amzn_norm)
    if eltype(amzn_norm[!, col]) <: Number
        amzn_norm[!, col] = min_max_scaling(amzn_norm[!, col])
    end
end

using Dates
dates = amzn_norm."Date"
print(dates)
adj_close = amzn_norm."Adj Close"
output_date_format = DateFormat("yyyy/mm/dd")
function parse_mixed_date(date_str)
    date_parsed = nothing
    try
        date_parsed = Date(date_str, DateFormat("m/d/yyyy"))
    catch
    end
    if isnothing(date_parsed)
        try
            date_parsed = Date(date_str, DateFormat("m/dd/yyyy"))
        catch
        end
    end
    return date_parsed
end
formatted_dates = [Dates.format(parse_mixed_date(date), 
    output_date_format) for date in dates]
Formatted_dates = Date.(formatted_dates, output_date_format)
init_date = Formatted_dates[1]
rel_dates = []
for x in Formatted_dates
    x = x - init_date
    push!(rel_dates, Dates.value(x))
end

using DataInterpolations
time_series = collect(zip(rel_dates, adj_close))
cubic_spline_t = DataInterpolations.CubicSpline(rel_dates, rel_dates)
cubic_spline_p = DataInterpolations.CubicSpline(adj_close, rel_dates)
function cubic_spline(t)
    return (cubic_spline_t(t), cubic_spline_p(t))
end